In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import plotly.express as px

# Data Exploration

In [2]:
data_1 = pd.read_csv("data/daily_financial_news/analyst_ratings_processed.csv", index_col=0)
data_1.head()

,title,date,stock
0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A


In [3]:
data_1['date'] = data_1['date'].str.split(' ', expand=True).iloc[:, 0]

In [4]:
data_1.head()

,title,date,stock
0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05,A
1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03,A
2.0,71 Biggest Movers From Friday,2020-05-26,A
3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22,A
4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22,A


In [5]:
data_1['stock'].nunique()

6192

In [6]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1400469 entries, 0.0 to 1413848.0
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   title   1400469 non-null  object
 1   date    1399180 non-null  object
 2   stock   1397891 non-null  object
dtypes: object(3)
memory usage: 42.7+ MB


## Drop Null Values

In [7]:
print(data_1.isna().sum())

title       0
date     1289
stock    2578
dtype: int64


In [8]:
print(data_1.shape)

(1400469, 3)


In [9]:
data_1[data_1['date'].isna()]

,title,date,stock
5714.0,Sterne Agee Provides Color on Aaron's,NaN,NaN
7021.0,Goldman Sachs Updates on Car Parts,NaN,NaN
13735.0,American Campus to Offer 15M Shares,NaN,NaN
13764.0,American Campus Communities Reports Q2 FFOM of...,NaN,NaN
14421.0,Positive Seeking Alpha Article on Aceto Corpor...,NaN,NaN
...,...,...,...
1410993.0,Zynga Loses Its CTO of Mobile -AllThingsD,NaN,NaN
1411126.0,Looks Like Zynga Just Bought OMGPOP For $200 M...,NaN,NaN
1411150.0,Zynga Lead Game Designer Leaving the Company -...,NaN,NaN
1412824.0,Waiting on Several IPOs This Morning:,NaN,NaN


In [10]:
data_1.dropna(subset=['date'],inplace=True)
data_1.dropna(subset=['stock'],inplace=True)

In [11]:
print(data_1.shape)

(1397891, 3)


In [12]:
print(data_1.isna().sum())

title    0
date     0
stock    0
dtype: int64


In [13]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1397891 entries, 0.0 to 1413848.0
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   title   1397891 non-null  object
 1   date    1397891 non-null  object
 2   stock   1397891 non-null  object
dtypes: object(3)
memory usage: 42.7+ MB


## Filter stocks

In [14]:
# Filter dataset down to stocks with top 100 number of headlines
top_100_stocks_by_headlines = data_1.groupby('stock').size().reset_index(name='size').sort_values('size', ascending=False).reset_index(drop=True).iloc[:100]
top_100_stocks_by_headlines.head()

,stock,size
0,MRK,3334
1,MS,3242
2,MU,3144
3,NVDA,3133
4,QQQ,3100


In [15]:
data_1 = data_1[data_1['stock'].isin(top_100_stocks_by_headlines.stock)]

In [16]:
data_1.shape

(212110, 3)

## Clean Text Data

In [17]:
data_1['title'] = data_1['title'].str.lower()

In [18]:
data_1.head()

,title,date,stock
0.0,stocks that hit 52-week highs on friday,2020-06-05,A
1.0,stocks that hit 52-week highs on wednesday,2020-06-03,A
2.0,71 biggest movers from friday,2020-05-26,A
3.0,46 stocks moving in friday's mid-day session,2020-05-22,A
4.0,b of a securities maintains neutral on agilent...,2020-05-22,A


### Remove Punctuations

In [19]:
import string

def remove_punctuation(text):
    return ''.join([char for char in text if char not in string.punctuation and not char.isdigit()])

data_1['title'] = data_1['title'].apply(remove_punctuation)

In [20]:
data_1.head()

,title,date,stock
0.0,stocks that hit week highs on friday,2020-06-05,A
1.0,stocks that hit week highs on wednesday,2020-06-03,A
2.0,biggest movers from friday,2020-05-26,A
3.0,stocks moving in fridays midday session,2020-05-22,A
4.0,b of a securities maintains neutral on agilent...,2020-05-22,A


In [21]:
data_1['tokens'] = data_1['title'].apply(lambda x: x.split())
data_1.head()

,title,date,stock,tokens
0.0,stocks that hit week highs on friday,2020-06-05,A,"[stocks, that, hit, week, highs, on, friday]"
1.0,stocks that hit week highs on wednesday,2020-06-03,A,"[stocks, that, hit, week, highs, on, wednesday]"
2.0,biggest movers from friday,2020-05-26,A,"[biggest, movers, from, friday]"
3.0,stocks moving in fridays midday session,2020-05-22,A,"[stocks, moving, in, fridays, midday, session]"
4.0,b of a securities maintains neutral on agilent...,2020-05-22,A,"[b, of, a, securities, maintains, neutral, on,..."


### Remove stop words

In [22]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop = stopwords.words('english')

data_1['tokens'] = data_1['tokens'].apply(lambda x: [word for word in x if word not in stop])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/weijie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
data_1.shape

(212110, 4)

### Lemmatize tokens

In [24]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

data_1['tokens'] = data_1['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


[nltk_data] Downloading package wordnet to /Users/weijie/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
data_1.head()

,title,date,stock,tokens
0.0,stocks that hit week highs on friday,2020-06-05,A,"[stock, hit, week, high, friday]"
1.0,stocks that hit week highs on wednesday,2020-06-03,A,"[stock, hit, week, high, wednesday]"
2.0,biggest movers from friday,2020-05-26,A,"[biggest, mover, friday]"
3.0,stocks moving in fridays midday session,2020-05-22,A,"[stock, moving, friday, midday, session]"
4.0,b of a securities maintains neutral on agilent...,2020-05-22,A,"[b, security, maintains, neutral, agilent, tec..."


In [26]:
data_1['preprocessed_text'] = data_1['tokens'].apply(' '.join)
data_1.head()

,title,date,stock,tokens,preprocessed_text
0.0,stocks that hit week highs on friday,2020-06-05,A,"[stock, hit, week, high, friday]",stock hit week high friday
1.0,stocks that hit week highs on wednesday,2020-06-03,A,"[stock, hit, week, high, wednesday]",stock hit week high wednesday
2.0,biggest movers from friday,2020-05-26,A,"[biggest, mover, friday]",biggest mover friday
3.0,stocks moving in fridays midday session,2020-05-22,A,"[stock, moving, friday, midday, session]",stock moving friday midday session
4.0,b of a securities maintains neutral on agilent...,2020-05-22,A,"[b, security, maintains, neutral, agilent, tec...",b security maintains neutral agilent technolog...


In [27]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
# NLTK Sentiment Intensity Analyzer uses a 'Bag of Words' approach
# it removes stop words and scores each word individually before compounding

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/weijie/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [28]:
data_duplicate = data_1.copy()
data_duplicate['sentiment_score'] = data_duplicate['preprocessed_text'].apply(lambda x: sia.polarity_scores(x)['compound'])
data_duplicate

,title,date,stock,tokens,preprocessed_text,sentiment_score
0.0,stocks that hit week highs on friday,2020-06-05,A,"[stock, hit, week, high, friday]",stock hit week high friday,0.0000
1.0,stocks that hit week highs on wednesday,2020-06-03,A,"[stock, hit, week, high, wednesday]",stock hit week high wednesday,0.0000
2.0,biggest movers from friday,2020-05-26,A,"[biggest, mover, friday]",biggest mover friday,0.0000
3.0,stocks moving in fridays midday session,2020-05-22,A,"[stock, moving, friday, midday, session]",stock moving friday midday session,0.0000
4.0,b of a securities maintains neutral on agilent...,2020-05-22,A,"[b, security, maintains, neutral, agilent, tec...",b security maintains neutral agilent technolog...,0.3400
...,...,...,...,...,...,...
1411294.0,is zyngas prethanksgiving ipo a turkey,2011-11-06,ZNGA,"[zyngas, prethanksgiving, ipo, turkey]",zyngas prethanksgiving ipo turkey,0.0000
1411295.0,zynga files new s stament for ipo cash on hand...,2011-11-04,ZNGA,"[zynga, file, new, stament, ipo, cash, hand, s...",zynga file new stament ipo cash hand slightly ...,0.4939
1411296.0,zynga ipo to debut week before thanksgiving ac...,2011-10-24,ZNGA,"[zynga, ipo, debut, week, thanksgiving, accord...",zynga ipo debut week thanksgiving according so...,0.0000
1411297.0,ipo filing zynga amends s filing,2011-10-14,ZNGA,"[ipo, filing, zynga, amends, filing]",ipo filing zynga amends filing,0.0000


In [29]:
data_duplicate['sentiment_score'].describe()

count    212110.000000
mean          0.072493
std           0.291640
min          -0.957800
25%           0.000000
50%           0.000000
75%           0.273200
max           0.974400
Name: sentiment_score, dtype: float64

In [30]:
data_duplicate.info()

<class 'pandas.core.frame.DataFrame'>
Index: 212110 entries, 0.0 to 1411298.0
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   title              212110 non-null  object 
 1   date               212110 non-null  object 
 2   stock              212110 non-null  object 
 3   tokens             212110 non-null  object 
 4   preprocessed_text  212110 non-null  object 
 5   sentiment_score    212110 non-null  float64
dtypes: float64(1), object(5)
memory usage: 11.3+ MB


In [31]:
data_duplicate.head()

,title,date,stock,tokens,preprocessed_text,sentiment_score
0.0,stocks that hit week highs on friday,2020-06-05,A,"[stock, hit, week, high, friday]",stock hit week high friday,0.00
1.0,stocks that hit week highs on wednesday,2020-06-03,A,"[stock, hit, week, high, wednesday]",stock hit week high wednesday,0.00
2.0,biggest movers from friday,2020-05-26,A,"[biggest, mover, friday]",biggest mover friday,0.00
3.0,stocks moving in fridays midday session,2020-05-22,A,"[stock, moving, friday, midday, session]",stock moving friday midday session,0.00
4.0,b of a securities maintains neutral on agilent...,2020-05-22,A,"[b, security, maintains, neutral, agilent, tec...",b security maintains neutral agilent technolog...,0.34


In [86]:
signals_df = data_duplicate[['date', 'stock', 'sentiment_score']].copy()

# Pull yfinance data

In [33]:
tickers = data_duplicate.stock.unique()

In [34]:
start_date, end_date = data_duplicate.date.sort_values().iloc[0], data_duplicate.date.sort_values().iloc[-1]

In [35]:
data = yf.download(list(tickers), start=start_date, end=end_date)

[*********************100%%**********************]  100 of 100 completed


13 Failed downloads:
['CHK', 'SHLD']: Exception("%ticker%: Data doesn't exist for startDate = 1240977600, endDate = 1591848000")
['BBBY', 'JCP', 'APC', 'MYL', 'ZNGA', 'AGN', 'MON']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BBRY', 'PCLN', 'GMCR', 'LNKD']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2009-04-29 -> 2020-06-11)')


In [87]:
adj_close_data = data['Adj Close']

In [88]:
# Tickers that don't have data
missing_data_tickers = adj_close_data.columns[adj_close_data.isna().sum()/adj_close_data.shape[0] == 1]

# Drop missing tickers
adj_close_data = adj_close_data.drop(columns=missing_data_tickers)

In [89]:
# Drop other tickers
adj_close_data = adj_close_data.dropna(axis=1)

In [90]:
any(adj_close_data.isna().sum() > 0)

False

In [91]:
returns_df = adj_close_data.pct_change().dropna().reset_index().rename(columns={'Date': 'date'})

In [92]:
returns_df_melt = returns_df.melt(id_vars='date', var_name='stock', value_name='daily_returns')
returns_df_melt['date'] = pd.to_datetime(returns_df_melt['date'])
returns_df_melt.head()

,date,stock,daily_returns
0,2009-04-30,A,0.012756
1,2009-05-01,A,0.030121
2,2009-05-04,A,0.032429
3,2009-05-05,A,-0.035530
4,2009-05-06,A,0.026695


In [93]:
signals_df.sort_values('date').head()

,date,stock,sentiment_score
1834.0,2009-04-29,A,0.0000
561526.0,2009-05-27,GMCR,0.7096
1833.0,2009-06-01,A,0.2960
794150.0,2009-06-16,M,0.8481
737639.0,2009-06-22,KO,0.8555


In [94]:
returns_df_melt.dtypes

date             datetime64[ns]
stock                    object
daily_returns           float64
dtype: object

In [95]:
signals_df['date'] = pd.to_datetime(signals_df['date'])

In [98]:
ml_df = pd.merge(returns_df_melt, signals_df, on=['date', 'stock'], how='left').dropna()

# Build Machine Learning Models

In [99]:
ml_df

,date,stock,daily_returns,sentiment_score
21,2009-06-01,A,0.035655,0.2960
51,2009-07-14,A,0.005157,0.3818
63,2009-07-30,A,0.023421,0.0000
66,2009-08-04,A,0.009186,0.0000
68,2009-08-06,A,-0.011049,0.0000
...,...,...,...,...
303402,2020-05-29,YUM,-0.012328,0.2960
303405,2020-06-03,YUM,0.025485,0.6908
303408,2020-06-08,YUM,-0.005491,0.0000
303410,2020-06-10,YUM,-0.015383,0.0000
